Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [2]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = './data/'

In [3]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.

Done.

- [ ] Continue to participate in our Kaggle challenge. 

Work below.

In [4]:
# Defining wrangle function. Copies matrix X to avoid setting with copy 
# warnings. Replaces outliers in latitude with 0. Replaces zeroes with NaN where 
# appropriate. Drops unusable variance. Essentially cleans matrix X with one
# function.

import numpy as np

def wrangle(X):

  X = X.copy()

  X['latitude'] = X['latitude'].replace(-2e-08, 0)

  cols_with_zeroes = ['longitude', 'latitude', 'construction_year',
                      'gps_height', 'population']

  for col in cols_with_zeroes:
    X[col] = X[col].replace(0, np.nan)
    X[col+'_missing'] = X[col].isnull()

  duplicates = ['quantity_group', 'payment_type']
  X = X.drop(columns=duplicates)

  random_noise = ['recorded_by', 'id']
  X = X.drop(columns=random_noise)
  
  return X

In [5]:
# Splitting train set into train/val sets.

from sklearn.model_selection import train_test_split

train, val = train_test_split(train, train_size=0.8, test_size=0.2,
                              stratify=train['status_group'], random_state=7)

train.shape, val.shape, test.shape

((47520, 41), (11880, 41), (14358, 40))

In [6]:
# Applying wrangle function to train/val/test sets.

train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [7]:
# Defining target and features to use. Dropping target because duh, excluding 
# high cardinality features.

target = 'status_group'

train_features = train.drop(columns=[target])

numeric_features = train_features.select_dtypes(include='number').columns.tolist()

cardinality = train_features.select_dtypes(exclude='number').nunique()

categorical_features = cardinality[cardinality <= 50].index.tolist()

features = numeric_features + categorical_features

In [8]:
# Breaking train/val/test sets into feature matrices and target matrices where
# appropriate. 

X_train = train[features]
y_train = train[target]

X_val = val[features]
y_val = val[target]

X_test = test[features]

In [9]:
pip install category_encoders

Note: you may need to restart the kernel to use updated packages.


In [10]:
# Importing all the stuff I'm probably going to use

import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [11]:
# Defining pipeline

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(n_jobs=-1, random_state=7)
)

In [13]:
# Getting cross_val_score of pipeline

k = 5
scores = cross_val_score(pipeline, X_train, y_train, cv=k,
                         scoring='accuracy')
print(f'Accuracy for {k} folds:', scores)

Accuracy for 5 folds: [0.79661195 0.7957702  0.8035564  0.80039983 0.79271886]


- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.

In [12]:
# Importing RandomizedSearchCV

from sklearn.model_selection import RandomizedSearchCV

In [13]:
# Defining parameter distributions

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'],
    'randomforestclassifier__max_depth': range(20, 30),
    'randomforestclassifier__min_samples_split': range(5, 10),
    'randomforestclassifier__min_samples_leaf': range(1, 5)
}

In [14]:
param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'],
    'randomforestclassifier__max_depth': range(20, 30),
    'randomforestclassifier__min_samples_split': range(6, 11),
    'randomforestclassifier__min_samples_leaf': range(1, 6),
    'randomforestclassifier__max_features': range(6, 14)
}

In [28]:
# Instantiating/defining instance of RandomizedSearchCV

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=40,
    cv=3,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

In [29]:
# Fitting search to training set

search.fit(X_train, y_train)

Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   28.0s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   39.2s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  7.2min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            fill_value=None,


In [30]:
print('Best hyperparameters:', search.best_params_)
print('Cross-validation Accuracy:', search.best_score_)

Best hyperparameters: {'simpleimputer__strategy': 'mean', 'randomforestclassifier__min_samples_split': 6, 'randomforestclassifier__min_samples_leaf': 2, 'randomforestclassifier__max_features': 6, 'randomforestclassifier__max_depth': 29}
Cross-validation Accuracy: 0.8005050505050505


In [15]:
rf = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    RandomForestClassifier(random_state=7, 
                           n_jobs=-1, 
                           n_estimators=300, 
                           min_samples_leaf=1, 
                           min_samples_split=9,
                           max_depth=25,
                           warm_start=True)
)
rf.fit(X_train, y_train)
print('Random Forest Train Accuracy Score:', rf.score(X_train, y_train))
print('Random Forest Validation Accuracy Score:', rf.score(X_val, y_val))

Random Forest Train Accuracy Score: 0.9050925925925926
Random Forest Validation Accuracy Score: 0.8178451178451178


In [16]:
rf_2 = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(random_state=7, 
                           n_jobs=-1, 
                           n_estimators=300, 
                           min_samples_leaf=2, 
                           min_samples_split=6,
                           max_depth=26,
                           warm_start=True)
)
rf_2.fit(X_train, y_train)
print('Random Forest Train Accuracy Score:', rf_2.score(X_train, y_train))
print('Random Forest Validation Accuracy Score:', rf_2.score(X_val, y_val))

Random Forest Train Accuracy Score: 0.8986742424242424
Random Forest Validation Accuracy Score: 0.8192760942760943


In [17]:
rf_3 = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    RandomForestClassifier(random_state=7, 
                           n_jobs=-1, 
                           n_estimators=300, 
                           min_samples_leaf=2, 
                           min_samples_split=6,
                           max_depth=27,
                           max_features=9,
                           warm_start=True)
)
rf_3.fit(X_train, y_train)
print('Random Forest Train Accuracy Score:', rf_3.score(X_train, y_train))
print('Random Forest Validation Accuracy Score:', rf_3.score(X_val, y_val))

Random Forest Train Accuracy Score: 0.9166877104377105
Random Forest Validation Accuracy Score: 0.8186868686868687


In [18]:
y_pred = rf_2.predict(X_test)

In [19]:
test_copy = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')

submission = test_copy[['id']].copy()
submission['status_group'] = y_pred

In [20]:
submission.to_csv('tanzania-waterpumps-submission-april-15-manalo-karl.csv', index=False)